In [14]:
import requests
import os, shlex
from pathlib import Path

# API Configuration
#try:
#    api_key = os.environ["LANGFLOW_API_KEY"]
#except KeyError:
#    raise ValueError("LANGFLOW_API_KEY environment variable not found. Please set your API key in the environment variables.")

api_key = 'sk-X79_-vNIayyYPiDeRRImuz71MV41uqitGS_IMjmLT5s'
url = "http://localhost:7861/api/v1/run/cfc72f63-b9d3-43b0-9559-220a9343d4ec"  # The complete API endpoint URL for this flow

In [15]:
defaults = {
    "PYTHON_SCRIPT": "/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reduce.py",
    "MOLECULES_CSV": "/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reactants_memory.csv",
    "REACTIONS_JSON": "/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/Reactions.json",
    "OUT_CSV": "/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reaction_products.csv",
}

for k, v in defaults.items():
    os.environ.setdefault(k, v)

# Quick sanity check (warn only)
for k in ("PYTHON_SCRIPT", "MOLECULES_CSV", "REACTIONS_JSON", "OUT_CSV"):
    p = Path(os.environ[k])
    if not p.exists():
        print(f"⚠️ {k} not found at {p}")

⚠️ MOLECULES_CSV not found at /media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reactants_memory.csv
⚠️ OUT_CSV not found at /media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reaction_products.csv


In [16]:
prompt = {
  "count": 5,
  "max_atoms": 8,
  "min_unique_elements": 1,
  "elements": "CNOF",

  "model_id": "ibm-research/GP-MoLFormer-Uniq",
  "tokenizer_id": "ibm-research/MoLFormer-XL-both-10pct",

  "temperature": 0.425,
  "top_p": 0.9,
  "top_k": None,
  "repetition_penalty": 1.10,
  "num_return_sequences": 96,

  "device": "cuda:0",
  "rdkit_filter": True,
  "remap_disallowed": True,

  "python_script": os.environ["PYTHON_SCRIPT"],
  "python_args": [
    "--molecules", os.environ["MOLECULES_CSV"],
    "--reactions", os.environ["REACTIONS_JSON"],
    "--out", os.environ["OUT_CSV"],
    "--depth", "2",
  ],

  "bash_cmd": "bash \"/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/run_conformer_search_nw_generation.sh\"",

  "memory_path": os.environ["MOLECULES_CSV"],
  "return_smiles": True
}

In [17]:
prompt

{'count': 5,
 'max_atoms': 8,
 'min_unique_elements': 1,
 'elements': 'CNOF',
 'model_id': 'ibm-research/GP-MoLFormer-Uniq',
 'tokenizer_id': 'ibm-research/MoLFormer-XL-both-10pct',
 'temperature': 0.425,
 'top_p': 0.9,
 'top_k': None,
 'repetition_penalty': 1.1,
 'num_return_sequences': 96,
 'device': 'cuda:0',
 'rdkit_filter': True,
 'remap_disallowed': True,
 'python_script': '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reduce.py',
 'python_args': ['--molecules',
  '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reactants_memory.csv',
  '--reactions',
  '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/Reactions.json',
  '--out',
  '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reaction_products.csv',
  '--depth',
  '2'],
 'bash_cmd': 'bash "/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/run_conformer_search_nw_generation.sh"',


In [18]:
# Request payload configuration
payload = {
    "output_type": "chat",
    "input_type": "chat",
    "input_value": str(prompt)
}

# Request headers
headers = {
    "Content-Type": "application/json",
    "x-api-key": api_key  # Authentication key from environment variable
}

In [19]:
try:
    # Send API request
    response = requests.request("POST", url, json=payload, headers=headers)
    response.raise_for_status()  # Raise exception for bad status codes

    # Print response
    print(response.text)

except requests.exceptions.RequestException as e:
    print(f"Error making API request: {e}")
except ValueError as e:
    print(f"Error parsing response: {e}")

{"session_id":"cfc72f63-b9d3-43b0-9559-220a9343d4ec","outputs":[{"inputs":{"input_value":"{'count': 5, 'max_atoms': 8, 'min_unique_elements': 1, 'elements': 'CNOF', 'model_id': 'ibm-research/GP-MoLFormer-Uniq', 'tokenizer_id': 'ibm-research/MoLFormer-XL-both-10pct', 'temperature': 0.425, 'top_p': 0.9, 'top_k': None, 'repetition_penalty': 1.1, 'num_return_sequences': 96, 'device': 'cuda:0', 'rdkit_filter': True, 'remap_disallowed': True, 'python_script': '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reduce.py', 'python_args': ['--molecules', '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reactants_memory.csv', '--reactions', '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/Reactions.json', '--out', '/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0_agent_construction/reaction_products.csv', '--depth', '2'], 'bash_cmd': 'bash \"/media/cameron/T7 Touch/Github/CGruich/RedoxFlow/scripts/0